# Basic Multi Antenna Raw File Generation
This tutorial walks through generation of Multi Antenna GUPPI RAW data with injected sine signals in Gaussian noise.

If you have access to a GPU, it is highly recommended to install CuPy, which performs the equivalent NumPy array operations on the GPU (https://docs.cupy.dev/en/stable/install.html). This is not necessary to run raw voltage generation, but will highly accelerate the pipeline. If you do install CuPy to use the GPU, it can be useful to run `export CUDA_VISIBLE_DEVICES=0` before running this notebook to specify a single GPU to use.

In [1]:
# !pip install cupy-cuda110

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from astropy import units as u
import blimpy as bl

import sys
sys.path.insert(0, "../../")
import setigen as stg

bshuf filter already loaded, skip it.
lz4 filter already loaded, skip it.
zfp filter already loaded, skip it.


In [3]:
# Sometimes it can be necessary to re-run this command for plots to show automatically
%matplotlib inline

We first set some basic parameters behind the pipeline. `sample_rate` is in samples per second (Hz); `num_taps` and `num_branches` are specific to the polyphase filterbank described below. 

In [4]:
sample_rate = 3e9
num_taps = 8
num_branches = 1024

chan_bw = sample_rate / num_branches

## Creating a MultiAntennaArray

We first create an `MultiAntennaArray` object, which initializes the array with `Antenna` instances each with an associated `delay` (in time samples). In addition to the individual data streams that allow you to add noise and signals to each `Antenna`, there are "background" data streams `bg_x` and `bg_y` in `MultiAntennaArray`, representing common / correlated noise or RFI that each `Antenna` can see, subject to the `delay`. (Note: `delays` can be `None` when initializing a `MultiAntennaArray`.)

In [5]:
delays = np.array([0, 1e-6, 2e-6]) * sample_rate
maa = stg.voltage.MultiAntennaArray(num_antennas=3,
                                    sample_rate=sample_rate,
                                    fch1=6*u.GHz,
                                    ascending=False,
                                    num_pols=2,
                                    delays=delays)

Let's add some Gaussian noise to the background streams, as well as a single "RFI" signal.

In [6]:
# This is equivalent to `for stream in [maa.bg_x, maa.bg_y]`
for stream in maa.bg_streams:
    stream.add_noise(v_mean=0,
                     v_std=1)
    stream.add_signal(f_start=5998.9e6, 
                      drift_rate=0*u.Hz/u.s, 
                      level=0.0025,
                      mode='level')

Adding data stream sources to each `Antenna`:

In [7]:
for stream in maa.antennas[0].streams:
    stream.add_noise(0, 1)

for stream in maa.antennas[1].streams:
    stream.add_noise(0, 2)
    stream.add_signal(f_start=5000.3e6, 
                      drift_rate=0*u.Hz/u.s, 
                      level=0.002,
                      mode='level')

for stream in maa.antennas[2].streams:
    stream.add_noise(0, 3)
    stream.add_signal(f_start=5000.7e6, 
                      drift_rate=0*u.Hz/u.s, 
                      level=0.004,
                      mode='level')

## Making the backend elements and recording data
As in the single `Antenna` version, we create the backend components according to desired parameters and construct the backend, this time passing in the `MultiAntennaArray` instead of a single `Antenna` object.

In [8]:
digitizer = stg.voltage.RealQuantizer(target_fwhm=32,
                                      num_bits=8)

filterbank = stg.voltage.PolyphaseFilterbank(num_taps=num_taps, 
                                             num_branches=num_branches)

requantizer = stg.voltage.ComplexQuantizer(target_fwhm=32,
                                           num_bits=8)

rvb = stg.voltage.RawVoltageBackend(maa,
                                    digitizer=digitizer,
                                    filterbank=filterbank,
                                    requantizer=requantizer,
                                    start_chan=0,
                                    num_chans=64,
                                    block_size=6291456,
                                    blocks_per_file=128,
                                    num_subblocks=32)

Actually "running" our recording:

In [9]:
rvb.record(raw_file_stem='/datax/scratch/bbrzycki/data/raw_files/example_multi',
           num_blocks=1, 
           length_mode='num_blocks',
           header_dict={'HELLO': 'test_value'},
           verbose=False)

Blocks: 100%|██████████| 1/1 [00:03<00:00,  3.12s/it]       
